Qui procederemo a creare un insieme di modelli che lavorano contemporaneamente producendo i loro output sulla base dei quali verrà applicata la legge della total variance.



In [29]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import statistics as stat


# Model class

In [ ]:
class CNNLSTMModel(nn.Module):

  def __init__(self, input_size, num_lstm_layers, dropout,
               sequence_length,nStepsAhead):
    super().__init__()
    self.jitter = 1e-6

    # model definition
    self.hybridNetwork = nn.Sequential(
        nn.Conv1d(in_channels=input_size,
                  out_channels=sequence_length,
                  kernel_size=1),
        nn.ReLU(),
        nn.LSTM(input_size=sequence_length,
                hidden_size=sequence_length,
                num_layers=num_lstm_layers,
                dropout=dropout,
                batch_first=False,bidirectional=True),
    )
    self.endLinearLayer = nn.Linear(2 * sequence_length,2 * nStepsAhead)

  def forward(self, x):
    # Apply CNN layers
    x = x.permute(0, 2, 1)
    x = self.hybridNetwork[0](x)  # Conv1d
    x = self.hybridNetwork[1](x)  # ELU activation

    # Permute the dimensions for LSTM
    x = x.permute(2, 0, 1)

    # Apply LSTM layers
    x, (hidden, cell) = self.hybridNetwork[2](x)

    # Only take the output from the last time-step
    x = x[-1, :, :]

    # Apply final linear layer
    x = self.endLinearLayer(x)
    means ,variances = x.chunk(2, dim = -1)
    variances = self.jitter + variances
    variances = torch.clamp(variances, min=1e-3)  # Enforce a minimum positive variance
    distributions = torch.distributions.Normal(means, variances)
    return distributions

class Model:
    def __init__(self, config):
        for index, row in config.iterrows():
            setattr(self, row['ParamName'], row['Value'])
        # una volta che sono tutti trainati e il campo link pth non sarà vuoto qui dovremmo usare initModel()
        self.model = 0

    def initModel(self):
      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      model = CNNLSTMModel(int(self.input_size), int(self.num_lstm_layers),float(self.dropout),int(self.window_size),int(self.nStepsAhead))
      model.load_state_dict(torch.load(self.model_path, map_location=device))
      model = model.to(device)
      self.model = model

## Configuration

In [66]:
configFilePath = "/content/drive/MyDrive/Colab Notebooks/NeuroProbabilistico/NeuroProb/ConfigModels.csv"
modelsPTHPath = "/content/drive/MyDrive/Colab Notebooks/NeuroProbabilistico/NeuroProb/NeuroProbPTH/NeuroProbModelsSet/ModelsSet.pth"

# read file and build model using configuration dict
configFile = pd.read_csv(configFilePath, delimiter=';')
dfList =  []

for colonna, serie in configFile.items():
  df = pd.DataFrame()
  if colonna == "Name":
      df[colonna] = serie
  else:
    values_list = []
    for elemento in serie:
        values_list.append(elemento)
    df["Value"] = values_list
  dfList.append(df)

for el in dfList:
  el["ParamName"] = configFile["Name"]
  tmp = el.pop('ParamName')
  el.insert(0, 'ParamName', tmp)

dfList.pop(0)
dfList[0].head(30)
modelList = [Model(el) for el in dfList]

CNNLSTMModel(
  (hybridNetwork): Sequential(
    (0): Conv1d(61, 176, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): LSTM(176, 176, num_layers=3, dropout=0.5256, bidirectional=True)
  )
  (endLinearLayer): Linear(in_features=352, out_features=48, bias=True)
)
